In [1]:
# default_exp ocrlearner
%load_ext autoreload
%autoreload 2

# OCR Learner

Simple OCR models for use with font optimiser. We make some spefcific adjustments for this case, including:

* Using grayscale images and, thus, 1-channel inputs
  * Note that this demands other changes as well: remove ImageNet normalisation routines
  * default.cmap to binary instead of viridis
* Cutting the Resnet 18 model at layer 5
* Balancing input categories

To do:

* Implement oversampling
* Implement normalisation

In [2]:
#export

from aifont.core import *
from aifont.fontsampler import FontSampler
import cornet
from fastai.data.all import *
from fastai.vision.all import *
from enum import Enum, auto
import gc
import hashlib
from itertools import product
from nbdev.showdoc import *
import matplotlib.pyplot as plt
import os
import pandas as pd
from pandas import DataFrame
import PIL
import random
import re
import scipy.io
import string
import torchvision.transforms.functional as VF
import torchvision.transforms as VT
from typing import Callable, List, Sequence, Tuple

## Data Sources

### Files and DataFrames

In [3]:
#export

A_Z_HANDW_PATH     = Path("data/A_Z_handwritten_data.csv")
DLS_TEST_PATH      = Path("data/az_tmnist_for_dls_test.csv")
GOOGLE_FONT_PATH   = Path("data/google_fonts/font_images")
MODELS_PATH        = Path("models")
NIST_PATH          = Path("data/nist_images")
STANFORD_PATH      = Path("data/letter.data.gz") 
TMNIST_PATH        = Path("data/94_character_TMNIST.csv")

def ascii_to_str(ascii: str) -> str:
    """Decode an ASCII-encoded character."""
    return bytes.fromhex(ascii).decode()

def nist_from_name_func(name: str) -> str:
    """Use with `ImageDataLoaders.from_name_func` for nist images."""
    code = name.split("_")[1] # E.g. train_59_01642.png
    return ascii_to_str(code)

def pad_to_size(
    df: DataFrame, 
    size: Union[int, tuple[int, int]] = None, # w = h or (w,h)
    start_col = 1,
    pad_with  = 0.
    ) -> DataFrame:
    """Pad the image data in DataFrame `df` with `pad_with` to match `size`.
       Note that column names are reset to integers with -1 for the ones added."""
    if size is None: return df
    nw,nh = (size,size) if type(size) is int else size
    init_sz = df.shape[1] - start_col
    if init_sz == nw * nh: return df
    ow = oh = math.isqrt(init_sz)
    assert ow * oh == init_sz, "Input images must be square"
    pw,ph = (nw - ow), (nh - oh)
    assert pw >= 0 and ph >= 0, "Size must be greater than equal to the original"
    assert pw % 2 == 0 and ph % 2 == 0, "Padding must be symmetrical in both dims"
    ndf = df.copy()
    ndf.columns = range(ndf.shape[1])
    cols = np.arange(start_col, start_col + init_sz).reshape((oh,ow))
    cols = np.pad(cols, ((ph//2,),(pw//2,)), constant_values=-1)
    cols = list(range(start_col)) + cols.ravel().tolist()
    return ndf.reindex(columns=cols, fill_value=pad_with)

def undersample_df(df: DataFrame, label_col_i = 0, undersample_threshold = 2., seed = 42) -> DataFrame:
    """Balances samples in categories to match the smallest one times `undersample_threshold`."""
    counts = df.iloc[:, label_col_i].value_counts()
    n_min = round(counts.min() * undersample_threshold)
    if (np.array(counts) <= n_min).all(): return df
    rs = random.getstate()
    random.seed(seed)
    keep = []
    tot = len(df)
    for i, c in counts.iteritems():
        ii = df.index[df.iloc[:, label_col_i] == i].tolist()
        keep.append(random.sample(ii, n_min) if c > n_min else ii)
    df = df.filter(items=[i for ii in keep for i in ii], axis=0)
    warn(f"Undersampled to max {n_min} items / category, using {len(df)} items of {tot}.")
    random.setstate(rs)
    return df

def get_a_z_handw_images(path=A_Z_HANDW_PATH, undersample = True, 
                         undersample_threshold = 2., seed = 42) -> DataFrame:
    """Get a collection of NIST and other images at 28x28.
       https://www.kaggle.com/datasets/sachinpatel21/az-handwritten-alphabets-in-csv-format"""
    df = pd.read_csv(path)
    if undersample: df = undersample_df(df, label_col_i=0, undersample_threshold=undersample_threshold, seed=seed)
    return df

def get_tmnist_images(path=TMNIST_PATH, relabel_vocab=VOCAB_UC, undersample = True, 
                      undersample_threshold = 2., seed = 42, drop_extra_cols = True) -> DataFrame:
    """Get a collection of renderend font glyphs at 28x28 and relabel with `relabel_vocab`
       for combination with `get_a_z_handw_images`.
       https://www.kaggle.com/datasets/nikbearbrown/tmnist-alphabet-94-characters"""
    df = pd.read_csv(path)
    def get_labels_num(label_str):
        label_str = str(label_str)
        return relabel_vocab.index(label_str) if label_str in relabel_vocab else np.nan
    labels_num = DataFrame([get_labels_num(x) for x in df.labels], columns=['labels_num'])
    df = pd.concat([labels_num, df], axis=1)
    df.drop(df[df.labels_num.isna()].index, inplace=True)
    if drop_extra_cols: df.drop(['names', 'labels'], axis=1, inplace=True)
    df.labels_num = df.labels_num.astype(int)
    if undersample: df = undersample_df(df, label_col_i=0, undersample_threshold=undersample_threshold, seed=seed)
    return df

def get_combined_az_and_tmnist_df(undersample = True, undersample_threshold = 2., 
                                  seed = 42) -> DataFrame:
    """Combined both the handwritten `get_a_z_handw_images` and `get_tmnist_images`.
       NB. These are not weighted much, just a bit for the us_th for tmnist."""
    df_az  = get_a_z_handw_images(undersample=undersample, undersample_threshold=undersample_threshold, seed=seed)
    df_tmnist = get_tmnist_images(undersample=undersample, undersample_threshold=undersample_threshold * .75, seed=seed)
    df_tmnist.columns = df_az.columns
    return pd.concat((df_az, df_tmnist))

def get_az_tmnist_for_dls_test() -> DataFrame:
    """A truncated dataset that's quick to load when testing `DataLoaders`"""
    df = pd.read_csv(DLS_TEST_PATH)
    return df.drop(columns=[df.columns[0]]) # Drop unnamed index
    
def get_nist_images(vocab: List[str] = None, path = NIST_PATH, show_stats = True, 
                    undersample = True, undersample_threshold = 2., seed = 42) -> List[Path]:
    """Get NIST handwritten images optionally filtered by `vocab`. If `undersample`, balances
       samples in categories to match the smallest one times `undersample_threshold`.
       National Institute of Standards and Technology: NIST Handprinted Forms and Characters
       - NIST Special Database 19, 2nd Edition, http://doi.org/10.18434/T4H01C (Accessed Apr 18, 2022)"""
    fnames = {}
    folders = os.listdir(path)
    for d in folders:
        c = ascii_to_str(d)
        if vocab is not None and c not in vocab: continue
        d_path = os.path.join(path, d)
        for d2 in os.listdir(d_path):
            if re.search("^train", d2): fnames[c] = get_image_files(os.path.join(d_path, d2))
    if show_stats:
        cc = sorted(fnames.keys())
        nn = [len(fnames[x]) for x in cc]
        plt.bar([i for i in range(len(cc))], nn, tick_label=cc)
    if undersample:
        rs = random.getstate()
        random.seed(seed)
        nn = [len(x) for x in fnames.values()]
        n_min = round(min(nn) * undersample_threshold)
        tot = sum(nn)
        for x in fnames: 
            if len(fnames[x]) > n_min: fnames[x] = random.sample(fnames[x], n_min)
        warn(f"Undersampled to max {n_min} items / category, using {sum([len(x) for x in fnames.values()])} items of {tot}.")
        random.setstate(rs)
    return [f for ff in fnames.values() for f in ff]

def stanford_letters_df(path = STANFORD_PATH) -> DataFrame:
    """from http://ai.stanford.edu/~btaskar/ocr/"""
    return pd.read_csv(path, delimiter="\t")

def get_font_df(
    font_paths: list[Union[str, Path]] = [SYS_FONT_PATH/"Arial.ttf"],
    font_size: float = 0.71,
    font_y: float = None,
    size: int = 28,
    vocab: list = VOCAB_UC
) -> DataFrame:
    """Create `DataFrame` by rendering the fonts."""
    data = []
    font_size = round(font_size * size)
    # Calculate a nice y position if we have an all-uppercase vocab
    if font_y is None: font_y = font_size + (size - font_size) // 2 if ''.join(vocab).isupper() else None
    else:              font_y = round(font_y * size)
    for font in font_paths:
        for i,l in enumerate(vocab):
            img = render_text(font, text=l, text_size=size, image_width=size, image_height=size, y=font_y, as_normalised_array=True)
            data.append([i] + list(255 * (1. - img.flatten())))
    df = pd.DataFrame(data).astype("uint8")
    df.columns = ["Letter_idx"] + [f"{i}_{j}" for i in range(size) for j in range(size)]
    return df

@delegates(get_font_df)
def get_fontsampler_df(
    variants: list[str] = None, 
    subsets: list[str] = None, 
    category: str = None, 
    **kwargs
) -> DataFrame:
    """Create a `DataFrame` by rendering fonts from `FontSampler`."""
    assert any([variants, subsets, category])
    return get_font_df(font_paths=FontSampler(variants=variants, subsets=subsets, category=category).paths, **kwargs)

@delegates(get_fontsampler_df)
def get_sans_serif_df(**kwargs) -> DataFrame: 
    """Create a `DataFrame` by rendering Latin sans-serif fonts from `FontSampler`."""
    return get_fontsampler_df(category="sans-serif", variants=["regular"], subsets=["latin"], **kwargs)

### Category Distributions

In [4]:
#export

class TensorCategoryDistribution(TensorCategory): pass

TensorBase.register_func(Tensor.__getitem__, TensorCategoryDistribution)

class CategoryDistribution(ShowTitle): 
    "A simple class for representing category distributions."
    _show_args = {'label': 'category'}
    def __init__(self, category: str, p: float): self.category,self.p = category,p
    def __str__(self): return f"{self.category} ({self.p:0.4f})"

class GetCategoryDistribution(DisplayedTransform):
    """Transform category string to propability distribution from `confusion_matrix`.
       Enforces a sum of one for each input's propabilities if `normalize`."""
    loss_func,order,y_tensors=CrossEntropyLossFlat(),1,{}
    def __init__(self, 
        confusion_matrix: DataFrame,
        vocab = None, 
        sort = True,
        normalize = True
        ):
        if vocab is not None: vocab = CategoryMap(vocab, sort=sort)
        store_attr()

    def setups(self, dsets):
        if self.vocab is None and dsets is not None: self.vocab = CategoryMap(dsets, sort=self.sort)
        self.c = len(self.vocab)
        for o in self.vocab:
            assert o in self.confusion_matrix.index
            t = TensorCategoryDistribution(self.confusion_matrix.loc[o])
            self.y_tensors[o] = t / t.sum() if self.normalize else t

    def encodes(self, o): 
        try:
            return self.y_tensors[o]
        except KeyError as e:
            raise KeyError(f"Label '{o}' was not included in the training dataset") from e
    
    def decodes(self, o):
        idx = o.argmax().item()
        return CategoryDistribution(self.vocab[idx], o[idx].item())

def CategoryDistibutionBlock(
    confusion_matrix: DataFrame,
    vocab = None, 
    sort = True,
    normalize = True
):
    "`TransformBlock` category propability distributions."
    return TransformBlock(type_tfms=GetCategoryDistribution(confusion_matrix=confusion_matrix, 
                          vocab=vocab, sort=sort, normalize=normalize))

### DataFrame DataLoader and Transforms

In [5]:
#export

class ImageReaderDF(DisplayedTransform):
    def __init__(self, 
        start_col = 1, 
        end_col: int = None, 
        height = 28, 
        width = 28,
        invert = True, 
        dtype = 'uint8'
        ):
        store_attr()

    def __call__(self, o, **kwargs):
        o = o[self.start_col : self.end_col if self.end_col is not None else len(o)] \
            .values.reshape(self.height, self.width).astype(self.dtype)
        return 255 - o if self.invert else o

class AlphabetClassReaderDF(DisplayedTransform):
    def __init__(self, 
        label_col = 0, # The column index in the df that holds the label or its index, if vocab is supplied
        vocab: List[str] = None, # The vocab
        ):
        store_attr()

    def __call__(self, o, **kwargs):
        l = o[self.label_col]
        return self.vocab[int(l)] if self.vocab else l

class ImageDataLoadersDF(ImageDataLoaders):
    @classmethod
    @delegates(DataLoaders.from_dblock)
    def from_df(cls, df, 
        valid_pct=0.2, 
        seed=None, 
        path=".",
        start_col=1, 
        end_col: int=None, 
        height=28, 
        width=28,
        label_col=0, 
        vocab: List[str]=None,
        y_block=None, 
        valid_col=None, 
        item_tfms=None, 
        batch_tfms=None, 
        color=False,
        **kwargs
        ):
        if y_block is None: y_block = CategoryBlock
        splitter = RandomSplitter(valid_pct, seed=seed) if valid_col is None else ColSplitter(valid_col)
        dblock = DataBlock(blocks=(partial(ImageBlock, cls=PILImage if color else PILImageBW), y_block),
                           get_x=ImageReaderDF(start_col=start_col, end_col=end_col, height=height, width=width),
                           get_y=AlphabetClassReaderDF(label_col=label_col, vocab=vocab),
                           splitter=splitter,
                           item_tfms=item_tfms,
                           batch_tfms=batch_tfms)
        return cls.from_dblock(dblock, df, path=path, **kwargs)

### Stats About the Data

Some stats on the data sources, which should be taken into account when choosing the rendering parameters for the vector optimiser.

In [6]:
#export

NIST_STATS = {
    'top_min': 2,
    'top_mean': 41.7465,
    'top_max': 58,
    'btm_min': 60,
    'btm_mean': 84.995,
    'btm_max': 127,
    'letter_height_min': 13,
    'letter_height_mean': 43.2485,
    'letter_height_max': 125,
    'width': 128,
    'height': 128
    }

AZ_STATS = {
    'top_min': 4,
    'top_mean': 4.5936,
    'top_max': 12,
    'btm_min': 15,
    'btm_mean': 22.4108,
    'btm_max': 27,
    'letter_height_min': 3,
    'letter_height_mean': 17.8172,
    'letter_height_max': 23,
    'width': 28,
    'height': 28
}
TMNIST_STATS = {
    'top_min': 0,
    'top_mean': 5.1545,
    'top_max': 29,
    'btm_min': -1,
    'btm_mean': 23.0986,
    'btm_max': 27,
    'letter_height_min': -30,
    'letter_height_mean': 17.9441,
    'letter_height_max': 20,
    'width': 28,
    'height': 28
}

def calc_nist_stats(n=10000):
    """Calc the tops and bottoms of the letters in the images."""
    fnames = get_nist_images(show_stats=False, undersample=False)
    img = PIL.Image.open(fnames[0])
    w, h = img.width, img.height
    tops, btms, lhs = [], [], []
    for f in random.sample(fnames, n):
        img = PIL.Image.open(f)
        img_a = np.array(img.getdata())[:,0].reshape(h, w, -1)
        top, btm = h + 1, -1
        for i in range(h):
            if (img_a[i] < 255/2).any():
                if i < top: top = i
                if i > btm: btm = i
        tops.append(top)
        btms.append(btm)
        lhs.append(btm - top)
    tops, btms, lhs = np.array(tops), np.array(btms), np.array(lhs)
    return {
        "top_min":  tops.min(),
        "top_mean": tops.mean(),
        "top_max":  tops.max(), 
        "btm_min":  btms.min(), 
        "btm_mean": btms.mean(),
        "btm_max":  btms.max(),
        "letter_height_min":  lhs.min(),
        "letter_height_mean": lhs.mean(),
        "letter_height_max":  lhs.max(),
        "width":    w,
        "height":   h,
        }

def calc_df_stats(n=10000, df=get_a_z_handw_images):
    """Calc the tops and bottoms of the letters in the images."""
    df = df()
    rows, cols = df.shape
    size = int((cols - 1) ** .5) # Assume one label col
    tops, btms, lhs = [], [], []
    for i in random.sample(list(range(rows)), n):
        top, btm = size + 1, -1
        for j in range(1, cols): # Skip first label col
            if df.iloc[i, j] > 255/2: # NB. Reverse values
                row_n = j // size
                if row_n < top: top = row_n
                if row_n > btm: btm = row_n
        tops.append(top)
        btms.append(btm)
        lhs.append(btm - top)
    tops, btms, lhs = np.array(tops), np.array(btms), np.array(lhs)
    return {
        "top_min":  tops.min(),
        "top_mean": tops.mean(),
        "top_max":  tops.max(), 
        "btm_min":  btms.min(), 
        "btm_mean": btms.mean(),
        "btm_max":  btms.max(),
        "letter_height_min":  lhs.min(),
        "letter_height_mean": lhs.mean(),
        "letter_height_max":  lhs.max(),
        "width":    size,
        "height":   size,
        }

## Augmentation

In [9]:
#export

# For use with `get_dls` later. These are the args used for matching
# the OCR model's accuracy to that of empirical studies, i.e., 50%
EMPIRICAL_FINETUNING_TFMS = dict(
    use_affine_tfms=False,
    use_xtra_tfms=True, 
    tfms_p=1.,
    blur_size=17,
    blur_sigma=4.41, 
    noise_f=None,
    size=48, 
    normalize=True,
    translate_and_pad=.2
)

# Another version of these tfms trained without noise-causing translate_and_pad
EMPIRICAL_FINETUNING_2_TFMS = dict(
    use_affine_tfms=False,
    use_xtra_tfms=True, 
    tfms_p=1.,
    blur_size=17,
    blur_sigma=5.0625, 
    noise_f=None,
    size=48, 
    normalize=True,
    translate_and_pad=0.
)

# Yet aother version of these tfms with 50% basic accuracy
EMPIRICAL_FINETUNING_3_TFMS = dict(
    use_affine_tfms=False,
    use_xtra_tfms=True, 
    tfms_p=1.,
    blur_size=21,
    blur_sigma=5.46875, 
    noise_f=None,
    size=48, 
    normalize=True,
    translate_and_pad=0.
)


class BlendingMethod(Enum):
    "For use with `add_noise`"
    MIX = auto()
    ADD = auto()
    SUB = auto()
BLENDING_METHODS = [o for o in BlendingMethod]

class NoiseType(Enum):
    "For use with `add_noise`"
    UNIFORM = auto()
    GAUSSIAN = auto()
NOISE_TYPES = [o for o in NoiseType]

class PaddingColor(Enum):
    "For use with `apply_gaussian_blur`"
    MAX = auto()
    MIN = auto()

def add_noise(x: TensorOrImage,
    method: BlendingMethod = None,
    noise_type: NoiseType = None,
    f: Union[float, Tensor, tuple, list] = None, # Fixed value, range as tuple or None for (0., 1.)
    clip = (0., 1.)
    ) -> Tensor:
    """Add random noise to the tensor."""
    assert is_pil(x) or torch.is_floating_point(x)
    if method is None:     method = random.choice(BLENDING_METHODS)
    if noise_type is None: noise_type = NoiseType.UNIFORM
    if f is None:     fac = random.random()
    elif is_listy(f): fac = f[0] + random.random() * (f[1] - f[0])
    else:             fac = f
    assert method in BLENDING_METHODS and noise_type in NOISE_TYPES
    assert noise_type != NoiseType.GAUSSIAN or method == BlendingMethod.ADD
    noise = torch.rand(x.shape) if noise_type == NoiseType.UNIFORM else .5 * torch.randn(x.shape)
    xt = VF.to_tensor(x) if is_pil(x) else x
    assert xt.max() <= 1.1, "Use image tensors in range (0, 1) for add_noise"
    if   method == BlendingMethod.MIX: xt = (1. - fac) * xt + fac * noise
    elif method == BlendingMethod.ADD: xt = xt + fac * noise
    else:                              xt = xt - fac * noise # BlendingMethod.SUB
    if clip is not None: xt = torch.clip(xt, *clip)
    return VF.to_pil_image(xt) if is_pil(x) else xt

def apply_gaussian_blur(x: TensorOrImage,
    gb: VT.GaussianBlur,
    pad_before = PaddingColor.MAX
    ) -> TensorOrImage:
    """Apply a `VT.GaussianBlur` with pre-padding to get rid artifacts at the edges."""
    kernel_size = gb.kernel_size
    xt = VF.to_tensor(x) if is_pil(x) else x
    if pad_before is not None:
        if   pad_before == PaddingColor.MAX: fill = math.ceil(xt.max())
        elif pad_before == PaddingColor.MIN: fill = math.floor(xt.min())
        else: raise ValueError(f"Unknown PaddingColor {pad_before}.")
        size = xt.shape[-2:]
        xt = VF.pad(xt,
                    padding=kernel_size,
                    fill=fill,
                    padding_mode="constant")
    xt = gb(xt)
    if pad_before is not None: xt = VF.crop(xt, kernel_size[1], kernel_size[0], *size)
    return VF.to_pil_image(xt) if is_pil(x) else xt

def rand_translate_and_pad(x: TensorOrImage,
    max_x = 0.1,
    max_y = 0.1,
    min_x = 0.,
    min_y = 0.,
    apply_to_batch = False
    ) -> TensorOrImage:
    """Translate the image tensor `x` with reflection padding by a random fraction 
       of the image size. If `apply_to_batch`, apply the same random values to the
       whole batch. No that without `apply_to_batch` the performance might be slow."""
    sz = list(x.shape[-2:])
    def _rnd_tl(): return (round((min_x + random.random() * (max_x - min_x)) * sz[0]),
                           round((min_y + random.random() * (max_y - min_y)) * sz[1]))
    if apply_to_batch or is_pil(x) or x.ndim < 4:  return translate_and_pad(x, *_rnd_tl())
    elif x.ndim == 4: return torch.concat([translate_and_pad(o, *_rnd_tl()) for o in x]).reshape((-1, *x.shape[-3:]))
    else: raise ValueError(f"Can't deal with {x.ndim} dimensions unless apply_to_batch is True.")

def translate_and_pad(x: TensorOrImage,
    tl_x: int = 0,
    tl_y: int = 0
    ) -> TensorOrImage:
    """Translate the image tensor `x` by `tl_x` and `tl_y` with reflection padding."""
    xt = VF.to_tensor(x) if is_pil(x) else x
    sz = list(xt.shape[-2:])
    assert sz[0] // 2 >= abs(tl_x) and sz[1] // 2 >= abs(tl_y)
    xt = VF.crop(xt,
                 top    = max(-tl_y, 0),
                 left   = max(-tl_x, 0),
                 height = sz[0] - abs(tl_y),
                 width  = sz[1] - abs(tl_x))
    xt = VF.pad(xt,
                padding=[
                     tl_x if tl_x > 0 else 0,
                     tl_y if tl_y > 0 else 0,
                    -tl_x if tl_x < 0 else 0,
                    -tl_y if tl_y < 0 else 0,
                ],
                padding_mode="reflect")
    return VF.to_pil_image(xt) if is_pil(x) else xt

def get_imagenet_norm() -> Normalize:
    """Perform ImageNet normalisation"""
    return Normalize.from_stats(mean=[0.485, 0.456, 0.406],
                                std=[0.229, 0.224, 0.225])

class Pad(DisplayedTransform):
    """Pad image or tensor. Can't use `CropPad` as it doesn't operate on `ImageTensor`.
       `size` is either h=v or (h,v)."""
    def __init__(self, 
        size: Union[int, Tuple[int, int]],
        fill: Union[int, float] = None
        ):
        super().__init__()
        if type(size) is int: size = (size,)*2
        store_attr()
    def encodes(self, x: TensorCategory): return x
    def encodes(self, x: TensorOrImage):
        return VF.pad(x, padding=self.size, fill=self.fill, 
                      padding_mode='constant' if self.fill is not None else 'symmetric')
    def encodes(self, x): raise ValueError(f"Pad cannot be applied to {type(x)}")

class RandImgTransform(RandTransform):
    """Perform one of `self.tfms` on an image or tensor with propability `p`."""
    def __init__(self, 
        p = 0.5, 
        tfms = [noop],
        **kwargs
        ):
        super().__init__(p=p, **kwargs)
        store_attr()
    def _encodes(self, x): return self.tfms[0](x) if len(self.tfms) == 1 else random.choice(self.tfms)(x)
    def encodes(self, x: Tensor):     return self._encodes(x) if is_image(x) else x
    def encodes(self, x: ImageTypes): return self._encodes(x)
    def encodes(self, x): raise ValueError(f"RandImgTransform cannot be applied to {type(x)}")

class GaussianBlur(RandImgTransform):
    """Gaussian blur with propability `p`, kernel `kernel_size` or `random_size`, and `sigma`."""
    def __init__(self, 
        p = 0.5, 
        kernel_size = (3, 3), 
        random_size: Union[int, Tuple[int, int]] = None,
        sigma = (0.1, 5), 
        pad_before = PaddingColor.MAX,
        **kwargs
        ):
        if random_size is not None:
            if type(random_size) is int: random_size = (1, random_size)
            assert is_listy(random_size)
            for o in random_size: assert o % 2 == 1
            kernel_size = product(range(random_size[0], random_size[1] + 2, 2), repeat=2)
        else: kernel_size = [kernel_size]
        tfms = []
        for o in kernel_size:
            if o == (1,1): continue
            gb = VT.GaussianBlur(kernel_size=o, sigma=sigma)
            tfms.append(partial(apply_gaussian_blur, gb=gb, pad_before=pad_before))
        super().__init__(p=p, tfms=tfms, **kwargs)
        store_attr()

class Noise(RandImgTransform):
    """Add noise with propability `p`, factor `f` and blending `method`. Leave to `None` to randomise."""
    def __init__(self, 
        p = 0.5, 
        f = None,
        method: BlendingMethod = None,
        noise_type: NoiseType = None,
        clip = (0., 1.),
        **kwargs
        ):
        clip = [float(o) for o in clip]
        tfms = [partial(add_noise, method=method, noise_type=noise_type, f=f, clip=clip)]
        super().__init__(p=p, tfms=tfms, **kwargs)
        store_attr()

class TranslateAndPad(RandImgTransform):
    """Translate with padding with propability `p` and random translation with the range specified as
       fractions of image size."""
    def __init__(self, 
        p = 0.5, 
        max_x = 0.1,
        max_y = 0.1,
        min_x = 0.,
        min_y = 0.,
        apply_to_batch = False,
        **kwargs
        ):
        assert all([max_x < .5, max_y < .5, min_x <= max_x, min_y <= max_y])
        tfms = [partial(rand_translate_and_pad, max_x=max_x, max_y=max_y, min_x=min_x, min_y=min_y, 
                                                apply_to_batch=apply_to_batch)]
        super().__init__(p=p, tfms=tfms, **kwargs)
        store_attr()

class ToRGB(DisplayedTransform):
    """Expand to 3 channels"""
    def encodes(self, x: Tensor): return tensor_to_rgb(x)
    def encodes(self, x: ImageTypes): return x.convert('RGB')
    def encodes(self, x): raise ValueError(f"ToRGB cannot be applied to {type(x)}")

class DebugTfm(DisplayedTransform):
    """Debug dataloaders"""
    log = []
    def __init__(self, msg = "", vocal = True):
        super().__init__()
        store_attr()
    def encodes(self, x):
        if self.vocal: print(f"{self.msg}: {type(x)} • {x.shape} • range: {x.min().item():.5f}-{x.max().item():.5f}"
                             f" • me(di)an {x.mean().item() if torch.is_floating_point(x) else x.median().item():.5f}")
        self.log.append(x)
        return x

## Deprecated: Forcing Fastai Vision Tools to Greyscale

This is a very hacky way to force images to grayscale! Note that we also need to disable normalisation at `cnn_learner`.

These are only needed for image data loaders that read image files. The dataframe-based ones can handle BW image creation themselves.

In [10]:
#export

PIL_BASE_ARGS = {}

def enable_global_greyscale():
    """Force all PILImages to greyscale. This is needed to force Fastai vision tools
       to greyscale. To undo, call `disable_global_greyscale`."""
    warn("Enabling global grayscale behaviour for PILImages! To undo, call disable_global_greyscale.")
    PIL_BASE_ARGS["_open_args"],PIL_BASE_ARGS["_show_args"] = PILBase._open_args,PILBase._show_args
    PILBase._open_args,PILBase._show_args = PILImageBW._open_args,PILImageBW._show_args

def disable_global_greyscale():
    """Undo forcing all PILImages to greyscale with `enable_global_greyscale`."""
    if len(PIL_BASE_ARGS): PILBase._open_args,PILBase._show_args = PIL_BASE_ARGS["_open_args"],PIL_BASE_ARGS["_show_args"]

# This could also be achieved with the following:

# def ImageBlockBW(cls=PILImageBW):
#     "A `TransformBlock` for images of `cls`"
#     return TransformBlock(type_tfms=cls.create, batch_tfms=IntToFloatTensor)

# def from_path_func_bw(path, fnames, label_func, valid_pct=0.2, seed=None, item_tfms=None, batch_tfms=None, **kwargs):
#     "Create from list of `fnames` in `path`s with `label_func`"
#     dblock = DataBlock(blocks=(ImageBlockBW, CategoryBlock),
#                         splitter=RandomSplitter(valid_pct, seed=seed),
#                         get_y=label_func,
#                         item_tfms=item_tfms,
#                         batch_tfms=batch_tfms)
#     return ImageDataLoaders.from_dblock(dblock, fnames, path=path, **kwargs)

# ImageDataLoaders.from_path_func = from_path_func_bw

## Model Architectures

### CORnet-Z

CORnet for letter recognition. We'll:

1. Use a model pretrained on ImageNet
2. Replace its head with one for letter categorisation
3. Freeze all but the last layer
4. Retrain with some augmentation and with letters within the centre 28x28px of a 48x48px input image
   1. We might want to compare this with inputs padded to 224x224px, which is the size used in pretraining

Ref: Kubilius, J., Schrimpf, M., Nayebi, A., Bear, D., Yamins, D.L.K., DiCarlo, J.J. (2018) CORnet: Modeling the Neural Mechanisms of Core Object Recognition. biorxiv. doi:10.1101/408385

In [11]:
#export

class CornetLayer(Enum):
    V1 = -5
    V2 = -4
    V4 = -3
    IT = -2
    decoder = -1

def cornet_for_ocr(n_out: int, split_at: CornetLayer = None, *args, **kwargs) -> Tuple[nn.Sequential, callable]:
    """Create a CORnet-Z model with outputs set to `n_out`."""
    model = cornet.cornet_z(pretrained=True, map_location=default_device())
    model = list(model.children())[0] # Get the Sequential from inside DataParallel
    old_lin = model[-1][2]
    new_lin = nn.Linear(old_lin.in_features, n_out)
    init_cnn(new_lin)
    model[-1][2] = new_lin
    splitter = cornet_splitter if split_at is None else partial(cornet_splitter, split_at=split_at)
    return model,splitter

def cornet_splitter(m, split_at: CornetLayer = CornetLayer.decoder): 
    "Splitter for CORnet-Z" 
    idx = split_at.value
    return L(m[:idx], m[idx]).map(params)

### Sample models from Kaggle submissions.

Recreating Keras models with Fastai.

In [17]:
#export

def count_params(model: nn.Module) -> int:
    """Count the number of trainable params in `model`."""
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

def count_max_pool_output(input_shape, ks=2, padding=0, stride=None):
    if stride is None: stride = ks
    return [(x + 2 * padding - (ks - 1) - 1) // stride + 1 for x in input_shape[:2]] + (input_shape[2:] if len(input_shape) > 2 else [])

def kaggle_cnn_a(n_out: int, input_shape=(28,28)):
    """https://www.kaggle.com/code/yairhadad1/cnn-for-handwritten-alphabets
       807,162 params"""
    # cls = Sequential()
    # cls.add(Conv2D(32, (5, 5), input_shape=(28, 28, 1), activation='relu'))
    # cls.add(MaxPooling2D(pool_size=(2, 2)))
    # cls.add(Dropout(0.3))
    # cls.add(Flatten())
    # cls.add(Dense(128, activation='relu'))
    # cls.add(Dense(len(y.unique()), activation='softmax'))
    ndim = 2
    filters = [32]
    ks = [5]
    n_pools = 1
    preflat_shape = tuple(input_shape)
    for _ in range(n_pools): preflat_shape = count_max_pool_output(preflat_shape)
    n_flat = int(np.prod(preflat_shape) * filters[-1])
    model = sequential(
        ConvLayer(1, filters[0], ks=ks[0], ndim=ndim),
        nn.Sequential(
            MaxPool(ndim=ndim),
            nn.Dropout(0.3),
            Flatten(),
            nn.Linear(n_flat, 128),
            nn.ReLU(),
            nn.Linear(128, n_out)
        )
    )
    init_cnn(model)
    return model

def kaggle_cnn_b(n_out: int, input_shape=(28,28)):
    """https://www.kaggle.com/code/codhek/cnn-using-keras-using-csv-accuracy-99-82
       1,132,250 params"""
    # model.add(Conv2D(64, (5, 5), input_shape=(28, 28, 1), activation='relu', data_format="channels_last", padding="same"))
    # model.add(Conv2D(64, (5, 5), input_shape=(28, 28, 1), activation='relu', data_format="channels_last", padding="same"))
    # model.add(MaxPooling2D(pool_size=(2, 2)))
    # model.add(Conv2D(128, (3, 3), activation='relu', data_format="channels_last", padding="same"))
    # model.add(Conv2D(128, (3, 3), activation='relu', data_format="channels_last", padding="same"))
    # model.add(MaxPooling2D(pool_size=(2, 2)))
    # model.add(Dropout(0.2))
    # model.add(Flatten())
    # model.add(Dense(128, activation='relu'))
    # model.add(Dense(num_classes, activation='softmax'))
    # NB. We are adding BatchNorm which comes with ConvLayer, use norm_type=None to disable
    ndim = 2
    filters = [64, 128]
    ks = [5, 3]
    n_pools = 2
    preflat_shape = tuple(input_shape)
    for _ in range(n_pools): preflat_shape = count_max_pool_output(preflat_shape)
    n_flat = int(np.prod(preflat_shape) * filters[-1])
    model = sequential(
        nn.Sequential(
            nn.Sequential(
                ConvLayer(1,          filters[0], ks=ks[0], ndim=ndim),
                ConvLayer(filters[0], filters[0], ks=ks[0], ndim=ndim),
            ),
            MaxPool(ndim=ndim),
            nn.Sequential(
                ConvLayer(filters[0], filters[1], ks=ks[1], ndim=ndim),
                ConvLayer(filters[1], filters[1], ks=ks[1], ndim=ndim),
            )
        ),
        nn.Sequential(
            MaxPool(ndim=ndim),
            nn.Dropout(0.2),
            Flatten(),
            nn.Linear(n_flat, 128),
            nn.ReLU(),
            nn.Linear(128, n_out)
        )
    )
    init_cnn(model)
    return model

def kaggle_cnn_c(n_out: int, input_shape=(28,28)):
    """https://www.kaggle.com/code/nohrud/99-9-accuracy-on-alphabet-recognition-by-eda
       594,170 params"""
    # model.add(Conv2D(32, (5, 5), input_shape=(28, 28, 1), activation='relu'))
    # model.add(MaxPooling2D(pool_size=(2, 2)))
    # model.add(Flatten())
    # model.add(Dense(128, activation='relu'))
    # model.add(Dense(num_classes, activation='softmax'))
    ndim = 2
    filters = [32]
    ks = [5]
    n_pools = 1
    preflat_shape = tuple(input_shape)
    # There's no padding for the first conv, so dims go down
    preflat_shape = [x - (ks[0] - 1) for x in preflat_shape]
    for _ in range(n_pools): preflat_shape = count_max_pool_output(preflat_shape)
    n_flat = int(np.prod(preflat_shape) * filters[-1])
    print(n_flat)
    model = sequential(
        nn.Sequential(
            ConvLayer(1, filters[0], ks=ks[0], ndim=ndim, padding=0),
        ),
        nn.Sequential(
            MaxPool(ndim=ndim),
            Flatten(),
            nn.Linear(n_flat, 128),
            nn.ReLU(),
            nn.Linear(128, n_out)
        )
    )
    init_cnn(model)
    return model
    
def simple_xresnet(n_out, **kwargs) -> Tuple[nn.Module, Callable]:
    """This matches XResNet18 and has 10M+ params, so it's huge..."""
    cut = -4
    model = XResNet(ResBlock, expansion=1, layers=[2,2,2,2], p=0.0, c_in=1, n_out=n_out, stem_szs=(32,32,64),
                    widen=1.0, ks=3, stride=2)
    # Cut into body and head as otherwise at least splitter will raise
    model = nn.Sequential(nn.Sequential(*list(model.children())[:cut]),
                          nn.Sequential(*list(model.children())[cut:]))
    def _xresnet_split(m): return L(m[0][:3], m[0][3:], m[1:]).map(params)
    return model, _xresnet_split

def kaggle_cnn_a_with_res(n_out: int, input_shape=(28,28)):
    """The Kaggle CNN A model with the first conv layer replaced by a res block
       https://www.kaggle.com/code/yairhadad1/cnn-for-handwritten-alphabets
       832,922 params"""
    ndim = 2
    filters = [32]
    ks = [5]
    n_pools = 1
    preflat_shape = tuple(input_shape)
    for _ in range(n_pools): preflat_shape = count_max_pool_output(preflat_shape)
    n_flat = int(np.prod(preflat_shape) * filters[-1])
    model = sequential(
        ResBlock(expansion=1,
                 ni=1,
                 nf=filters[0], 
                 stride=1,
                 ks=ks[0], 
                 ndim=ndim),
        nn.Sequential(
            MaxPool(ndim=ndim),
            nn.Dropout(0.3),
            Flatten(),
            nn.Linear(n_flat, 128),
            nn.ReLU(),
            nn.Linear(128, n_out)
        )
    )
    init_cnn(model)
    return model

## Create and load OCR Learners and models

In [18]:
#export

def list_params(learn: Learner) -> None:
    """Quickly list param groups and params to see which ones are frozen."""
    for i,g in enumerate(learn.opt.param_groups):
        for j,p in enumerate(g["params"]):
            print(f"Group {i} • Param {j}: {p.shape}, {'grad' if p.requires_grad else 'no grad'}")

def load_ocr_model(
    file: str = None, 
    arch: Callable = None, 
    df: Callable = None, 
    vocab: List[str] = VOCAB_UC,
    version: str = None,
    model_dir: str = MODELS_PATH, 
    n_out: int = None, 
    **kwargs
    ) -> Module:
    """Load a trained OCR model for use with `OCRLoss`. Pass either `file` and `arch` or
       `arch`, `df`, `version` and `vocab` to construct the filename automatically."""
    assert None not in (file, arch) or None not in (arch, df, vocab)
    if file is None: file = get_learner_filename(arch, df, vocab, version=version, incl_suffix=True)
    if n_out is None: n_out = len(vocab)
    file = os.path.join(model_dir, file)
    report(f"Loading OCR model: {file}")
    # This is awful, sry!
    model = arch(n_out=n_out, **kwargs)
    model = model[0] if type(model) is tuple else model
    load_model(file, model, opt=False, device=None)
    return model

def get_learner_filename(arch: Callable, df: Callable, 
    vocab: List[str] = None, 
    version: str = None,
    incl_suffix = False) -> str:
    """Create a filaneme for the model for use with saving and loading."""
    sfx = ".pth" if incl_suffix else ""
    vcb = f"__vocab_{hashlib.md5(''.join(vocab).encode('utf-8')).hexdigest()}" if vocab else ""
    ver = "" if version in (None, "") else f"_{version}"
    return f"{arch.__name__}__{df.__name__}{vcb}{ver}{sfx}"

def get_tfms(
    orig_sz:int,
    size=28,
    normalize=False, 
    tfms_p=.5,
    use_affine_tfms=True,
    use_xtra_tfms=False,
    blur_size:Union[int,Sequence[int]]=5,
    blur_sigma:Union[int,Sequence[int]]=(.1, 5.), 
    noise_f:Tuple[float,float]=(0., .6),
    noise_type=None,
    noise_method=None,
    translate_and_pad=0.,
    override_max_rotate=None,
    override_max_warp=None,
) -> Tuple[list, list]:
    "Build `tfms` and `item_tfms` for use with `ImageDataLoadersDF.from_df`."
    tfms,item_tfms = [],[]
    if orig_sz < size: 
        pad = (size - orig_sz) // 2
        assert orig_sz + 2 * pad == size
        item_tfms += [Pad(pad, fill=255)]
    if normalize: tfms += [get_imagenet_norm()] # ToRGB(), 
    if use_xtra_tfms:
        max_rotate = 15.0 if override_max_rotate is None else override_max_rotate
        max_warp   =  .25 if override_max_warp is None else override_max_warp
        if translate_and_pad > 0:
            item_tfms += [TranslateAndPad(p=tfms_p, max_x=translate_and_pad, max_y=translate_and_pad)]
        if blur_sigma or blur_size:
            size_args = dict(random_size=blur_size) if is_listy(blur_size) else dict(kernel_size=(blur_size,)*2)
            item_tfms += [GaussianBlur(p=tfms_p, sigma=blur_sigma, **size_args)]
        if noise_f:
            item_tfms += [Noise(p=tfms_p, f=noise_f, noise_type=noise_type, method=noise_method)]
    else: 
        max_rotate = 5.0 if override_max_rotate is None else override_max_rotate
        max_warp   =  .1 if override_max_warp is None else override_max_warp
    if use_affine_tfms:
        aug_tfms = aug_transforms(mult=1.0, do_flip=False, flip_vert=False, max_rotate=max_rotate, 
                    min_zoom=0.85, max_zoom=1.15, max_warp=max_warp, p_affine=tfms_p, 
                    p_lighting=0., xtra_tfms=None, size=size, mode='bilinear', 
                    pad_mode='reflection', align_corners=True, batch=False, 
                    min_scale=1.0)
        del(aug_tfms[1]) # Remove lighting tfm
        tfms += aug_tfms
    return tfms,item_tfms

@delegates(get_tfms)
def get_dls(
    df:Union[Callable, DataFrame], 
    bs=128, 
    start_col=1,
    vocab=VOCAB_UC,
    y_block=None, 
    seed=None,
    valid_pct=0.2,
    **kwargs
) -> DataLoaders:
    "Build `DataLoaders`."
    # Get DataFrame
    data = df() if callable(df) else df
    # Check bs doesn't exceed data
    max_bs = round(len(data) * valid_pct) if valid_pct > 0 else len(data)
    if bs > max_bs: bs = max_bs
    # Check if we need to pad
    orig_sz = math.isqrt(data.shape[1] - start_col)
    # Build tfms
    tfms,item_tfms = get_tfms(orig_sz=orig_sz, **kwargs)
    # tfms,item_tfms=None,None
    # Creata DataLoaders
    dls = ImageDataLoadersDF.from_df(data, vocab=vocab,
                                     width=orig_sz, height=orig_sz,
                                     num_workers=0, # Needed for Mac
                                     valid_pct=valid_pct, # Only training
                                     batch_tfms=tfms,
                                     item_tfms=item_tfms,
                                     color=False, #normalize,
                                     y_block=y_block,
                                     seed=seed, bs=bs)
    return dls

@delegates(get_dls)
def build_ocr_learner(
    model:Module=None, 
    splitter:Callable=cornet_splitter,
    arch:Callable=cornet_for_ocr,
    loss_func=CrossEntropyLossFlat(), 
    opt_func=Adam, 
    init=nn.init.kaiming_normal_,
    lr=defaults.lr, 
    cbs=None, 
    metrics=accuracy, 
    path=None, 
    model_dir=MODELS_PATH, 
    wd=None, 
    wd_bn_bias=False, 
    train_bn=True, 
    moms=(0.95,0.85,0.95),
    **kwargs
) -> Learner:
    """Create a new OCR learner based on `model` and `splitter` or an architecture
       generated by `arch` and dataset `df` (included in `**kwargs`)."""
    dls = get_dls(**kwargs)
    if model is None:
        n_out = get_c(dls)
        # This is a crappy, but we can specify the splitter in arch
        model_and_splitter = arch(n_out, input_shape=(dls.height, dls.width))
        if type(model_and_splitter) is tuple: model,splitter = model_and_splitter
        else: model,splitter = model_and_splitter,lambda m: L(m[0], m[1:]).map(params)
    else: assert splitter is not None
    learn = Learner(dls=dls, model=model, loss_func=loss_func, opt_func=opt_func, lr=lr, 
                    splitter=splitter, cbs=cbs,  metrics=metrics, path=path, model_dir=model_dir, 
                    wd=wd, wd_bn_bias=wd_bn_bias, train_bn=train_bn, moms=moms)
    return learn

Deprecated factory functions for backward compatibility.

In [ ]:
#export

def get_ocr_learner_2(
    arch=kaggle_cnn_a, 
    n_out=None, 
    bs=128, 
    df=get_a_z_handw_images, 
    vocab=VOCAB_UC,
    version=None,
    normalize=False, 
    load_saved=True, 
    size=28,
    loss_func=CrossEntropyLossFlat(), 
    opt_func=Adam, 
    init=nn.init.kaiming_normal_,
    lr=defaults.lr, 
    splitter=None, 
    use_xtra_tfms=False,
    cbs=None, 
    metrics=accuracy, 
    path=None, 
    model_dir=MODELS_PATH, 
    wd=None, 
    wd_bn_bias=False, 
    train_bn=True, 
    moms=(0.95,0.85,0.95), 
    **kwargs) -> Learner:
    """Create a new OCR learner based on architecture `arch` and dataset `df`.
       If `pretrained`, the model will be initialised if a suitable save is found."""
    if use_xtra_tfms:
        max_rotate = 15.0
        max_warp = .25
        blur = GaussianBlur(p=.5, random_size=5)
        noise = Noise(p=.5, f=(0., .6))
        item_tfms = [blur, noise]
    else: 
        max_rotate = 5.0
        max_warp = .1
        item_tfms = None
    tfms = aug_transforms(mult=1.0, do_flip=False, flip_vert=False, max_rotate=max_rotate, 
                          min_zoom=0.85, max_zoom=1.15, max_warp=max_warp, p_affine=.5, 
                          p_lighting=0., xtra_tfms=None, size=size, mode='bilinear', 
                          pad_mode='reflection', align_corners=True, batch=False, 
                          min_scale=1.0)
    del(tfms[1]) # Remove lighting tfm
    warn("Need to add custom normalisation scheme and check this doesn't expand channels!")
    # See: Normalize.from_stats(mean,std) https://docs.fast.ai/data.transforms.html#Normalize
    dls = ImageDataLoadersDF.from_df(df(), vocab=vocab,
                                     width=size, height=size,
                                     num_workers=0, # Needed for Mac
                                     valid_pct=0.2, 
                                     batch_tfms=tfms,
                                     item_tfms=item_tfms,
                                     seed=42, bs=bs)

    if n_out is None: n_out = get_c(dls)
    # This is a crappy, but we can specify the splitter in arch
    model_and_splitter = arch(n_out, input_shape=(size, size))
    if type(model_and_splitter) is tuple:
        model, splitter = model_and_splitter
    else: 
        model = model_and_splitter
        splitter = lambda m: L(m[0], m[1:]).map(params)
    learn = Learner(dls=dls, model=model, loss_func=loss_func, opt_func=opt_func, lr=lr, 
                    splitter=splitter, cbs=cbs,  metrics=metrics, path=path, model_dir=model_dir, 
                    wd=wd, wd_bn_bias=wd_bn_bias, train_bn=train_bn, moms=moms)
    if load_saved: learn.load(get_learner_filename(arch=arch, df=df, vocab=vocab, version=version))
    store_attr('arch,normalize,n_out', self=learn, **kwargs)
    return learn

def get_ocr_learner_3(
    arch=kaggle_cnn_a, 
    n_out=None, 
    bs=128, 
    df=get_combined_az_and_tmnist_df, 
    start_col=1,
    vocab=VOCAB_UC,
    version=None,
    normalize=False, 
    load_saved=True, 
    size=28,
    loss_func=CrossEntropyLossFlat(), 
    opt_func=Adam, 
    init=nn.init.kaiming_normal_,
    lr=defaults.lr, 
    splitter=None, 
    use_xtra_tfms=False,
    tfms_p=.5,
    blur_size: Tuple[int, Sequence[int]]=5,
    blur_sigma: Tuple[int, Sequence[int]]=(.1, 5.), 
    cbs=None, 
    metrics=accuracy, 
    path=None, 
    model_dir=MODELS_PATH, 
    wd=None, 
    wd_bn_bias=False, 
    train_bn=True, 
    moms=(0.95,0.85,0.95),
    seed=None,
    **kwargs) -> Learner:
    """Create a new OCR learner based on architecture `arch` and dataset `df`.
       If `pretrained`, the model will be initialised if a suitable save is found."""
    tfms = []
    item_tfms = []
    # Check if we need to pad
    data = df()
    orig_sz = math.isqrt(data.shape[1] - start_col)
    if orig_sz < size: 
        pad = (size - orig_sz) // 2
        assert orig_sz + 2 * pad == size
        item_tfms += [Pad(pad, fill=255)]
    if normalize: tfms += [get_imagenet_norm()] # ToRGB(), 
    if use_xtra_tfms:
        max_rotate = 15.0
        max_warp = .25
        blur = GaussianBlur(p=tfms_p, random_size=blur_size, sigma=blur_sigma)
        noise = Noise(p=tfms_p, f=(0., .6))
        item_tfms += [blur, noise]
    else: 
        max_rotate = 5.0
        max_warp = .1
    aug_tfms = aug_transforms(mult=1.0, do_flip=False, flip_vert=False, max_rotate=max_rotate, 
                          min_zoom=0.85, max_zoom=1.15, max_warp=max_warp, p_affine=tfms_p, 
                          p_lighting=0., xtra_tfms=None, size=size, mode='bilinear', 
                          pad_mode='reflection', align_corners=True, batch=False, 
                          min_scale=1.0)
    del(aug_tfms[1]) # Remove lighting tfm

    tfms += aug_tfms
    dls = ImageDataLoadersDF.from_df(data, vocab=vocab,
                                     width=orig_sz, height=orig_sz,
                                     num_workers=0, # Needed for Mac
                                     valid_pct=0.2, 
                                     batch_tfms=tfms,
                                     item_tfms=item_tfms,
                                     color=normalize,
                                     seed=seed, bs=bs)
    if n_out is None: n_out = get_c(dls)
    # This is a crappy, but we can specify the splitter in arch
    model_and_splitter = arch(n_out, input_shape=(size, size))
    if type(model_and_splitter) is tuple:
        model,splitter = model_and_splitter
    else: 
        model = model_and_splitter
        splitter = lambda m: L(m[0], m[1:]).map(params)
    learn = Learner(dls=dls, model=model, loss_func=loss_func, opt_func=opt_func, lr=lr, 
                    splitter=splitter, cbs=cbs,  metrics=metrics, path=path, model_dir=model_dir, 
                    wd=wd, wd_bn_bias=wd_bn_bias, train_bn=train_bn, moms=moms)
    if load_saved: learn.load(get_learner_filename(arch=arch, df=df, vocab=vocab, version=version))
    store_attr('arch,normalize,n_out', self=learn, **kwargs)
    return learn

## Calibrating Augmentations for 50% Accuracy

See notebook 14: OCR Learner Training 2: Match With Human Empirical Data.

In [ ]:
#export

@delegates(build_ocr_learner)
def get_model_accuracy_metrics(
    model:Module, 
    df:DataFrame,
    iters=1,
    show_batch=True,
    return_iters=False,
    **kwargs
) -> Union[Tuple[Learner, float, dict, float,  dict], 
           Tuple[Learner, float, dict, float,  dict, Tensor, Tensor]]:
    """Build a Learner and get accuracy metrics for it. Use `iters` to repeat 
       if dataset is small and using random augmentations. The tuple returned
       contains a copy of the `Learner` used, a mean of the propabilites for
       the correct class, a dict of these for each category, basic accurary,
       and basic accuracies for each class. If `return_iters` is `True`, the
       full correct class probabilies and target class `Tensors` are also
       returned."""
    learn = build_ocr_learner(model, df=df, valid_pct=0., **kwargs)
    if show_batch: learn.dls.train.show_batch()
    preds_l,trgts_l = [],[]
    with learn.no_logging():
        for _ in range(iters):
            preds,trgts = (cast(o, Tensor) for o in learn.get_preds(ds_idx=0))
            preds_l.append(preds); trgts_l.append(trgts)
    preds,trgts = concat_tensors(preds_l),concat_tensors(trgts_l)
    trgt_preds = preds.gather(1, trgts.unsqueeze(1)).squeeze()
    vocab = learn.dls.vocab[0]
    prob_dict = {}
    acc_dict = {}
    for i,l in enumerate(vocab):
        mask = (trgts == i).nonzero().squeeze()
        prob_dict[l] = trgt_preds.gather(0, mask).mean().item()
        acc_dict[l]  = accuracy(preds.index_select(0, mask), torch.full_like(mask, i)).item()
    prob_mean = trgt_preds.mean().item()
    acc_mean = accuracy(preds, trgts)
    out = (learn,prob_mean,prob_dict,acc_mean,acc_dict)
    return out + (trgt_preds,trgts) if return_iters else out

@delegates(get_model_accuracy_metrics)
def find_tfms_for_accuracy(
    model:Module, 
    df:DataFrame,
    target_acc=.5,
    target_prob=None,
    eps=.01,
    max_iters=40,
    param="blur_size",
    param_start=5,
    param_step=2,
    param_step_factor:float=0.5,
    base_tfms:dict=None,
    **kwargs
) -> Tuple[Learner,dict]:
    """Find values for tfms to pass to `get_tfms` that result in a mean accuracy of
       `target_acc` +/- `eps`, or `target_prob` which is the softmaxed probability
       of the target class. Note that by default we do not include affine rotation,
       scaling etc. transformations, nor noise. We only change blur."""
    assert param in ("blur_size", "blur_sigma", "noise_f")
    assert (target_acc is None) is not (target_prob is None)
    if param == "blur_size": 
        assert param_start % 2 == 1 and param_step % 2 == 0
        param_step_factor = math.ceil(param_step_factor)
    tfms_args = dict(
        use_affine_tfms=False,
        use_xtra_tfms=True, 
        tfms_p=1.,
        blur_size=5,
        blur_sigma=3., 
        noise_f=None,
        noise_type=None,
        noise_method=None,
        size=48, 
        normalize=True,
        translate_and_pad=0.
    )
    if base_tfms: tfms_args = {**tfms_args, **base_tfms}
    tfms_args[param] = param_start
    acc_mean = 1e9
    step = param_step
    target = target_prob if target_prob is not None else target_acc
    i = 0
    while abs(acc_mean - target) > eps and i < max_iters:
        if i > 0:
            # Param should be raised but it's negative or it should be lowered but it's positive,
            # i.e., on the last iter we went past the target. Thus, reverse the sign of the step
            # and lower its abs value.
            if (acc_mean > target and step < 0) or (acc_mean < target and step > 0):
                if param_step_factor == 1: print("No more param options available."); break
                step *= -1 * param_step_factor
            tfms_args[param] += step                
        res = get_model_accuracy_metrics(model, df, show_batch=False, **tfms_args, **kwargs)
        acc_mean,acc_dict = res[3:5] if target_prob is None else res[1:3]
        print(f"Iter {i} • Param ({param}): {tfms_args[param]} • Step {step} • Accuracy {acc_mean:.4f}")
        i += 1
    print("Final accuracy per letter")
    for l,a in acc_dict.items(): print(f"{l}: {a:.3f}")
    learn = res[0]
    learn.dls.show_batch()
    return learn,tfms_args

## Confusion Matrices: Comparing and OCR Model to Empirical Data

However, taking into account the critique offered in van der Heijden et al. (1984), the difference in stimulus fonts in Townsend (1971, typewriter) and Gilmore et al. (1979, dot matrix), the low number of observations in Phillips et al. (1983, 60 per letter), and the low number of test subjects in Loomis (1982, 6 female subjects), we use only the data from van der Heijden et al. (1984).

See `data/Letter confusion matrices combined.xlsx` for all of the above and further information.

**Note** that in, e.g., van def Heijden's data, the most probably response for the stimulus 'Q' was 'O'! This may be look confusing when using `show_batch` later.

In [ ]:
#export

CM_FOLDER_PATH     = Path("data")/"confusion_matrices"
CM_VD_HEIJDEN_PATH = CM_FOLDER_PATH/"van_der_heijden_et_al_1984.csv"
TOWNSEND_PATH      = CM_FOLDER_PATH/"townsend_1971_cond_1.csv"

# To get the other matrices, export as csv from the Excel file, but delete
# sum columns and rows. You may also need to convert decimal points.

def plot_confusion_matrix(
    learn: Learner, 
    change_loss_func = False
) -> None:
    """Utility for plotting a classical confusion matrix. NB. The other
       cm functions deal with ones based on class distributions."""
    loss = learn.loss_func
    if change_loss_func: learn.loss_func = CrossEntropyLossFlat()
    interp = ClassificationInterpretation.from_learner(learn)
    interp.plot_confusion_matrix(figsize=(12,12), dpi=60)
    if change_loss_func: learn.loss_func = loss

def get_empirical_confusion_matrix(
    path: Union[str, Path] = CM_VD_HEIJDEN_PATH,
    symmetrize=False,
) -> DataFrame:
    """Load a saved empirical confusion matrix and format it a bit. Optionally
       `symmetrize` off-diagonal values."""
    cm = pd.read_csv(path)
    cm.rename(columns={cm.columns[0]: "Stimulus"}, inplace=True)
    cm.set_index("Stimulus", drop=True, inplace=True)
    if symmetrize: cm = (cm + cm.T) / 2
    return cm

def get_confusion_matrix(
    ocr_fn: str,
    n_out = 26,
    arch = cornet_for_ocr,
    splitter = cornet_splitter,
    df = get_sans_serif_df,
    tfms = EMPIRICAL_FINETUNING_TFMS,
    ds_idx = 0,
    iters = 1
) -> DataFrame:
    """Compute a confusion matrix from output propabilites, not mere misclassifications."""
    ocr_model = load_ocr_model(file=ocr_fn, arch=arch, n_out=n_out)
    learn = build_ocr_learner(
        model=ocr_model, 
        splitter=splitter,
        df=df,
        loss_func=CrossEntropyLossFlat(flatten=False),
        **tfms
        )
    # Force augmentations to also work on the validation dataset, because `RandImgTfms` 
    # are only applied to the training dataset by default.
    for o in learn.dls.valid.after_item: o.split_idx = None
    vocab = learn.dls.vocab[0]
    preds_by_class = [[] for _ in vocab]
    for _ in range(iters):
        preds,targets = learn.get_preds(ds_idx=ds_idx)
        for p,c in zip(preds,targets): preds_by_class[c.item()].append(p)
    for i,p in enumerate(preds_by_class):
        preds_by_class[i] = [vocab[i]] + [o.item() for o in torch.stack(p).mean(dim=0)]
    cm = DataFrame(preds_by_class)
    cm.columns = ["Stimulus"] + vocab
    cm.set_index("Stimulus", inplace=True, drop=True)
    return cm

def compare_confusion_matrices(
    a: DataFrame,
    b: DataFrame,
    symmetrize=False,
    non_diagonal=False,
) -> DataFrame:
    """Calculate the Pearson correlation between two confusion matrices. For results
       comparable with traditional studies, set both `symmetrize` and `non_diagonal`
       to `True`."""
    from scipy.stats import pearsonr
    if symmetrize: a,b = (a + a.T) / 2, (b + b.T) / 2
    if non_diagonal:
        a_nd,b_nd = [],[]
        for i in range(len(a)): 
            a.iloc[i,i],b.iloc[i,i] = 0,0
            a_i,b_i = a.iloc[i].values.tolist(),b.iloc[i].values.tolist()
            del(a_i[i],b_i[i])
            a_nd.append(a_i); b_nd.append(b_i)
        index = a.index.tolist()
        a,b = DataFrame(a_nd),DataFrame(b_nd)
        a.index,b.index = index,index
    if symmetrize:
        # Remove duplicate entries before calculating total R
        lists = [[],[]]
        for i,o in enumerate([a,b]):
            for j in range(len(o)): 
                k = j if non_diagonal else j+1
                lists[i].append(o.values[j,:k].tolist())
        flat_r = pearsonr(*[flatten_list(o) for o in lists])
    else: 
        flat_r = pearsonr(*[flatten_list(o.values.tolist()) for o in (a,b)])
    per_cat_r = [("Total",) + flat_r]
    for c in a.index:
        per_cat_r.append((c,) + pearsonr(*[o.loc[c].values.tolist() for o in (a,b)]))
    per_cat_r = pd.DataFrame(per_cat_r)
    per_cat_r.columns = ["Stimulus", "Pearson R", "p"]
    per_cat_r.set_index("Stimulus", drop=True, inplace=True)
    return per_cat_r

## To Do: Oversampling

In [19]:
#hide

# It would be nice to have an oversampling method ready, but the code below is for Fast.ai v1
# and would have to be converted to probably something tied to the DataLoader.

from torch.utils.data.sampler import WeightedRandomSampler

class OverSamplingCallback(Callback):
    """Oversampling to correct unbalanced dataset.
       By Tanishq Mathew Abraham, https://www.kaggle.com/code/tanlikesmath/oversampling-mnist-with-fastai/notebook"""
    def __init__(self,learn:Learner):
        super().__init__(learn)
        ds = self.learn.data.train_dl.dataset
        self.labels = ds.y.items
        _, counts = np.unique(self.labels, return_counts=True)
        self.weights = torch.DoubleTensor((1/counts)[self.labels])
        self.label_counts = np.bincount([self.learn.data.train_dl.dataset.y[i].data \
                                         for i in range(len(ds))])
        self.total_len_oversample = int(self.learn.data.c * np.max(self.label_counts))
        
    def before_train(self, **kwargs):
        self.learn.data.train_dl.dl.batch_sampler = BatchSampler(WeightedRandomSampler(self.weights, self.total_len_oversample), 
                                                                 self.learn.data.train_dl.batch_size,
                                                                 False)

## Export

In [3]:
#hide
from nbdev.export import notebook2script; notebook2script()

Converted 00_ocr_model_training_1.ipynb.
Converted 01_fontlearnertests.ipynb.
Converted 02_first_letter_and_optim_tests.ipynb.
Converted 03_vector_models.ipynb.
Converted 04_font_statistics.ipynb.
Converted 05_full_aphabet_optimisation.ipynb.
Converted 06_data_augmentation.ipynb.
Converted 07_optimising_with_augmentation.ipynb.
Converted 08_comparing_existing_fonts.ipynb.
Converted 09_experiment_1_optimise_to_match_font.ipynb.
Converted 10_optimising_with_aug_run_2.ipynb.
Converted 11_optimise_to_match_font_multires.ipynb.
Converted 12_optimising_with_aug_with_multiple_ocr_models.ipynb.
Converted 13_analysing_ocr_models.ipynb.
Converted 14_experiment_2_ocrlearner_training_with_emp_data.ipynb.
Converted 15_experiment_3_optimise_with_emp_model.ipynb.
Converted 16_experiment_2_rerun.ipynb.
Converted 17_experiment_3_rerun.ipynb.
Converted aifont_core.ipynb.
Converted aifont_fontlearner.ipynb.
Converted aifont_fontsampler.ipynb.
Converted aifont_ocrlearner.ipynb.
Converted index.ipynb.
